In [27]:
from lxml import html
import requests
import pandas as pd 
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import subprocess as sp
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
import argparse
import subprocess

#hi I made this program and called it Daryl , Daryl is a good boy , he uses many toys such as web scrapping using beautiful soup
#and xpath to scrap the RWTH site , looks for informations needed , make trim the names , the adresses ,and store them in a
#pandas Dataframe , fill them up in my application form and send them up to the jobs stored in the Dataframe
#####!!!!!!!I'm open to critisizes so please let me know if something could be better!!!!!#########

#this function fills the pdf , fill the informations such as the institute name and adress 
#in variable file in latex wich is connected to the application form latex file
def fill_and_send (df): 
    df_done_list=[]
    for index, row in df.iterrows() :
        a=get_the_cover_pdf(row['Institute name'],row['Institute adress'])
        subject=('Bewerbung für die stelle "Studentische hilfskraft" von der '+row['Institute name'])
        body =  '''\
        <html>
          <head></head>
          <body>
            <p>Sehr geehrte Herr '''+row['Name_mod']+''', <br><br>
               Im anhang finden Sie meine Bewerbungsunterlagen für die oben genannte Stelle sowie meine Notenspiegel<br>
               <p style="text-align: right;">Ich freue mich sehr auf Ihren Rückmeldung</p><br>
               <p style="text-align: right;"><sub><u>Generated by:Daryl the Bewerbung Slave ver 1.0</u></sub></p><br>
            <a href="https://github.com/khalil3958/Daryl-the-Bewerbung-slave-ver-1-0.git">https://github.com/khalil3958/Daryl-the-Bewerbung-slave-ver-1-0.git</a>
            </p>
          </body>
        </html>
        '''
        initial_repertory={'Found keyword':row['Found keyword'],'Name':row['Name'],'Mail':row['Mail'],'Institute adress':row['Institute adress'],'Institute name':row['Institute name'],'Link':row['Link']}
        try :
            send_mail(row['Mail'],subject,body,a)
            df_done_list.append(initial_repertory)
        except : 
            pass
    df_done=pd.DataFrame(df_done_list)
    df_done.to_csv(r'C:\Users\hp\Downloads\beworben.csv')
    return df_done


#####################latex python interaction functions and pdf generation##########

def generate_cover_letter():
    os.chdir("C:\\Users\\hp\\Desktop\\ndhif\\test\\data\\")
    os.system("lualatex --interaction=nonstopmode coverletter.tex")


def get_the_cover_pdf(institute,adress):
    
    content = r'''\usepackage[utf8]{inputenc}
    \newcommand{\institute}{%(INSTITUTE)s}
    \newcommand{\adressie}{%(ADRESS)s}'''
    args=dict()
    args['INSTITUTE']=institute
    args['ADRESS']=adress
    with open('var.tex','w',encoding='utf8') as f:
        f.write(content%args)
    cmd = ['lualatex', '-interaction', 'nonstopmode', 'var.tex']
    proc = subprocess.Popen(cmd)
    proc.communicate()
    generate_cover_letter()
    pdf_path=['coverletter.pdf','cv.pdf']
    return pdf_path

############mail sender function##
def send_mail(receiver,subject,body,file_list):

    msg=MIMEMultipart('alternative')
    sender = 'khalil.jlif@rwth-aachen.de'
    msg['From']=sender
    msg['To']=receiver
    msg['Subject']=subject
    
    
    msg.attach(MIMEText(body,'html'))

    for filename in file_list:

        attachement=open(filename,'rb')
        part=MIMEBase('application','octet-stream')
        part.set_payload(attachement.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',"attachement; filename= "+ filename)
        msg.attach(part)

        text=msg.as_string()
    server=smtplib.SMTP('mail.rwth-aachen.de',587 )
    server.starttls()

    server.login('nm741271@rwth-aachen.de','my_password')
    server.sendmail(sender,receiver,text)
    server.quit()

########data_frame_generator_functions#########
    
def get_rwth_root(number_pages):
    roots=[]
    page=1
    
    while (page<=number_pages):
        print("page",page,"from",number_pages)
        rwth_url=requests.get('https://www.rwth-aachen.de/cms/root/Die-RWTH/Arbeiten-an-der-RWTH/~buym/Jobboerse/?page='+str(page))
        rwth_text=rwth_url.content
        mensa_soup=BeautifulSoup(rwth_text)
        link_in_page = []
        for a in mensa_soup.findAll("a",{'target':'_self'}):
            root =(a.get('href')) 
            if 'Einzelansicht'in root:
                link_in_page.append(root)
                table=(a.string).upper()
                if ("STUDENTISCHE HILFSKRAFT" in table):
                    roots.append(root)
                    
        if len(link_in_page) == 0:
                break
        page=page+1
    return roots


def check_keywords(stringewi , key_words):
        keywords_list=[] 
        big_string=stringewi.upper()
        for i in key_words :
            if i in big_string : 
                keywords_list.append(i)
        return keywords_list

def name_trim (stringie) : 
    aaa=stringie
    to_trim=to_trim=['M. Sc.','Dipl.-Wirt.-Ing.','M.Sc.','Dipl.-Phys. ',' M.Sc',' MSc']
    for i in to_trim:
        if i in stringie :
            aaa=stringie.replace(i,'')
            if 'RWTH' in aaa :
                aaa=aaa.replace('RWTH','')
    return aaa


def rwth_bewerbung_data(a,key_words):
    
    my_roots=get_rwth_root(a)
    repertory_list=[]

    for i in range (len(my_roots)):
        
        print("loading",int(i/len(my_roots)*100),"%")
        my_root_string=(my_roots[i])
        rwth_url_jobs=requests.get(my_root_string)
        rwth_url_text=rwth_url_jobs.content
        rwth_url_text_path=html.fromstring(rwth_url_text)
        soup=BeautifulSoup(rwth_url_text)

        for a in soup.findAll("div",{"class":"text"}):
            
            to_search=str(a.findAll('p'))
            found_keywords=check_keywords(to_search,key_words)
            
            if (len(found_keywords)==0):
                break
            #subject_name=a.find('h2').contents[0]
            betreuer_name=a.find('p',{'class':'fn n'}).string
            betreuer_mailie=rwth_url_text_path.xpath('//div[@id="main"]//dd[@class="value"]//a/@href')
            betreuer_mailiee=''.join(betreuer_mailie)
            betreuer_mail=betreuer_mailiee.replace('mailto:','')
            adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[3]//td[2]/text()'
            adress=rwth_url_text_path.xpath(adress_xpath)

            if len(adress)==0 or len(adress)==1 : 
                
                adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[2]//td[2]/text()'
                adress=rwth_url_text_path.xpath(adress_xpath)

            institut_name=rwth_url_text_path.xpath('//div[@id="main"]//div[@class="text"]//p[1]/text()')[-1]
            initial_repertory={'Found keyword':' '.join(found_keywords),'Name':betreuer_name,'Mail':betreuer_mail,'Institute adress':''.join(adress),'Institute name':institut_name,'Link':my_root_string}
            repertory_list.append(initial_repertory)
    
    bewerbung_infos=pd.DataFrame(repertory_list)
    bewerbung_infos['Name_mod'] = bewerbung_infos.apply(lambda x:name_trim(x['Name']),axis=1)
    return bewerbung_infos

key_words=['PYTHON','MATLAB','C++','C#','JAVA','ELEKTROTECH','INFORMATIK','REGELUNG','WEB','MODELLIER','MESSTECH','DATA','DIGITALIS']
fill_and_send(rwth_bewerbung_data(5,key_words))

page 1 from 5
page 2 from 5
page 3 from 5
page 4 from 5
page 5 from 5
loading 0 %
loading 0 %
loading 1 %
loading 2 %
loading 3 %
loading 4 %
loading 5 %
loading 6 %
loading 7 %
loading 8 %
loading 9 %
loading 10 %
loading 11 %
loading 11 %
loading 12 %
loading 13 %
loading 14 %
loading 15 %
loading 16 %
loading 17 %
loading 18 %
loading 19 %
loading 20 %
loading 21 %
loading 22 %
loading 22 %
loading 23 %
loading 24 %
loading 25 %
loading 26 %
loading 27 %
loading 28 %
loading 29 %
loading 30 %
loading 31 %
loading 32 %
loading 33 %
loading 33 %
loading 34 %
loading 35 %
loading 36 %
loading 37 %
loading 38 %
loading 39 %
loading 40 %
loading 41 %
loading 42 %
loading 43 %
loading 44 %
loading 44 %
loading 45 %
loading 46 %
loading 47 %
loading 48 %
loading 49 %
loading 50 %
loading 51 %
loading 52 %
loading 53 %
loading 54 %
loading 55 %
loading 55 %
loading 56 %
loading 57 %
loading 58 %
loading 59 %
loading 60 %
loading 61 %
loading 62 %
loading 63 %
loading 64 %
loading 65 %
loadi

,Found keyword,Name,Mail,Institute adress,Institute name,Link
0,ELEKTROTECH,Paul Buske,paul.buske@tos.rwth-aachen.de,Paul BuskeLehrstuhl für Technologie Optischer ...,Lehrstuhl für Technologie Optischer Systeme,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
1,INFORMATIK WEB DATA,Markus Grief,grief@wi.rwth-aachen.deavci@wi.rwth-aachen.de,LuF WirtschaftsinformatikHerr Markus GriefTemp...,LuF Wirtschaftsinformatik,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
2,MATLAB,Paul Pramod Muthyala,muthyala_p@vka.rwth-aachen.de,Lehrstuhl für Verbrennungskraftmaschinenz. Hd....,Lehrstuhl für Verbrennungskraftmaschinen (VKA),http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
3,PYTHON,Marvin Berger,marvin.berger@ipt.fraunhofer.de,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
4,MATLAB C++ MODELLIER,Charukeshi Joglekar,charukeshi.joglekar@eonerc.rwth-aachen.de,Charukeshi JoglekarInstitute for Automation of...,Automation of Complex Power Systems,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
5,REGELUNG MESSTECH,M. Sc. Jörg Hofmann,joerg.hofmann@tos.rwth-aachen.de,Herr Jörg HofmannLehrstuhl für Technologie Opt...,Lehrstuhl für Technologie Optischer Systeme,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
6,ELEKTROTECH INFORMATIK MESSTECH,Dr.-Ing. Christopher Brandl,c.brandl@iaw.rwth-aachen.de,Institut für ArbeitswissenschaftDr.-Ing. Chris...,Institut für Arbeitswissenschaft,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
7,INFORMATIK DATA DIGITALIS,Dipl.-Wirt.-Ing. Thomas Vollmer,studierende.produktionsqualitaet@ipt.fraunhofe...,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
8,WEB,Sebastian Weber M.Sc.,bewerbung.technologieorganisation@ipt.fraunhof...,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
9,C++ INFORMATIK,"Filippos Tzanetos, M.Sc.",filippos.tzanetos@ipt.fraunhofer.de,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
